In [107]:
from crawlbase import CrawlingAPI
from bs4 import BeautifulSoup
import pickle
import pandas as pd
import time

def scrape(sub_red):
    token = 'dLN3qq4_CCZeNJTAPTwGdA'
    reddit_url = 'https://www.reddit.com/r/' + sub_red + '/new/'
    api = CrawlingAPI({'token': token})
    response = api.get(reddit_url)

    if response['status_code'] == 200:
        return response['body']
    else:
        print('Error')
html_data = scrape('depression')

In [127]:
def save_post(html_data,sub_red):    
    with open('model.p', 'rb') as file:
        model = pickle.load(file)
    with open('vectorizer.p', 'rb') as file:
        vectorizer = pickle.load(file)
    soup = BeautifulSoup(html_data, 'html.parser')
    instances = soup.find_all('div', property="schema:articleBody")
    docs = []
    for instance in instances:
        for p_tag in instance:
            if p_tag:
            # print(''.join(p_tag.get_text().strip()))
                docs.append(''.join(p_tag.get_text().strip()))
    print(docs)

    verdicts = pd.read_csv('verdicts.csv')
    for doc in docs:
        vectorized = vectorizer.transform([doc])
        predict_probas = model.predict_proba(vectorized)[:, 1]
        threshold = 0.5
        bool = (predict_probas >= threshold)
        if not (verdicts['post'] == doc).any() and doc:
            verdicts = pd.concat([verdicts, pd.DataFrame({'post':doc,'verdict':bool, 'sub_red':sub_red})])
            verdicts.to_csv('verdicts.csv', index=False)
            print('saved a post from r/' + sub_red + ' ' + str(bool))
save_post(html_data,'depression')



['', 'As someone who’s gone through depression in the past and still faces it partly today, i’ve always found a great way of working through it was just talking. Thought i’d put this out there so if anyone ever wanted to talk privately they could, i’m always here for yall 🫶', '', '', "I don't know what would help besides killing me blowing my brains out all over the wall but I just want it to stop I can't handle living anymore 27 years was enough I want off stop it please", '', '', 'I am 27m. I have been battling gambling addiction for a while. I always tell myself i would stop. This time i fucked up bad.', "I was moneyless cause i gambled away my paycheck. It was going well, i didn't need any money so it was fine. Then, the other day, i got fined while driving. I asked for a cash advance on my paycheck, i wanted a small sum (paycheck is due in less than a week anyway), and my boss gave me almost all of it instead. I gambled it away. Again.", "I wanted to use that money to go on vacati

In [101]:
sub_reds = ['depression', 'sad', 'confessions']
sub_reds = ['sad']
iterations = 0
for sub_red in sub_reds:
    save_post(scrape(sub_red),sub_red)
    # time.sleep(10)
iterations += 1

saved a post from
saved a post from
saved a post from
saved a post from
saved a post from
saved a post from
saved a post from


In [125]:
verdicts = pd.DataFrame({'sub_red': sub_red,'verdict':[], 'post':[]})
verdicts.to_csv('verdicts.csv')

KeyboardInterrupt: 